In [2]:
import pandas as pd
import os
import json
import glob
from shapely.geometry  import shape, Point
from geopy.geocoders import Nominatim
from geopy import distance
import numpy as np

In [3]:
geolocator = Nominatim(user_agent="test_tk")

In [4]:
bigdf = pd.read_csv('./results/5_years_merged_with_weather_and_311.csv')

In [5]:
POLICE_DF = pd.read_csv('./Police_Stations_20240120.csv')

In [64]:
def find_closest(lat, long):
    if lat == 0 or long == 0:
        return 'No Lat/Long', 999999
    else:
        try:
            input_latlong = (long,lat)
            #distance_list = []
            pdf = POLICE_DF.copy()
            pdf['distance'] = pdf.apply(lambda x: distance.distance(input_latlong, (x.LONGITUDE,x.LATITUDE)).miles, axis=1)
            nearest_station = pdf.iloc[pdf['distance'].idxmin()]
            return nearest_station['station'], nearest_station['distance']
            """for row in POLICE_DF.iterrows():
                distance_holder = {}
                station_latlong = [row['LATITUDE'], row['LONGITUDE']]
                distance_holder['station'] = row['ADDRESS']
                distance_holder['distance'] = distance.distance(input_latlong, station_latlong).miles

                distance_list.append(distance_holder)
                distance_df = pd.DataFrame(distance_list)
                nearest_station = distance_df.iloc[distance_df['distance'].idxmin()]
                print(f"{distance_df.head()}")
                return nearest_station['station'], nearest_station['distance'] """
        except e:
            return 'Error something something', 999999

In [58]:
nearest_station = pdf.iloc[pdf['distance'].idxmin()]

In [59]:
minidf = bigdf.iloc[0:100]

In [62]:
minidf[['longitude','latitude']]

,longitude,latitude
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,-87.634761,41.818171
...,...,...
95,-87.646050,41.828347
96,-87.726359,41.865029
97,-87.736363,41.918245
98,-87.692793,41.896592


In [65]:
find_closest(-87.634761,41.818171)

('Error something something', 999999)

In [60]:
minidf[['closest_station', 'closest_station_distance']] = minidf.apply(lambda x: find_closest(x.longitude, x.latitude), axis=1, result_type='expand')

C:\Users\tkkim\AppData\Local\Temp\ipykernel_47276\3493688740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minidf[['closest_station', 'closest_station_distance']] = minidf.apply(lambda x: find_closest(x.longitude, x.latitude), axis=1, result_type='expand')
C:\Users\tkkim\AppData\Local\Temp\ipykernel_47276\3493688740.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  minidf[['closest_station', 'closest_station_distance']] = minidf.apply(lambda x: find_closest(x.longitude, x.latitude), axis=1, result_t

In [61]:
minidf['closest_station']

0     Error something something
1     Error something something
2     Error something something
3     Error something something
4     Error something something
                ...            
95    Error something something
96    Error something something
97    Error something something
98    Error something something
99    Error something something
Name: closest_station, Length: 100, dtype: object

In [34]:
#bigdf[['closest_station', 'closest_station_distance']] = bigdf[bigdf['latitude'].isnull() == False].apply(lambda x: find_closest(x.longitude, x.latitude), axis=1, result_type='expand')

In [35]:
bigdf.to_csv('./results/5_years_merged_with_weather_and_311and_policedistance.csv', index=False)

In [7]:
test = pd.read_csv('./results/5_years_merged_with_weather_and_311and_policedistance.csv')

In [8]:
test['closest_station'].value_counts()

3510 S Michigan Ave    1164132
Name: closest_station, dtype: int64

In [9]:
test.shape

(1181139, 136)